In [76]:
import os
from collections import defaultdict

cwd = os.getcwd()
file = cwd + '/inputs/input16.txt'

data = open(file).read().splitlines()
time_limit = 30


def parse_input(data):
  valves = []
  for line in data:
    valve = { "name": line.split()[1]}
    valve["value"] = int(line.split(';')[0].split('=')[1])
    if line.find('valves') == -1:
      valve["tunnels"] = [line.split('valve')[1].strip()]
    else:
      tunnels = line.split('valves')[1].split(',')
      valve["tunnels"] = []
      for tunnel in tunnels:
        valve["tunnels"].append(tunnel.strip())
        
    valves.append(valve)
  return valves

def find_important_nodes(valves):
  important_nodes = ['AA']
  for valve in valves:
    if valve["value"] != 0:
      important_nodes.append(valve["name"])
  return important_nodes

def create_graph(valves):
  graph = {}
  for valve in valves:
    graph[valve["name"]] = []
    for tunnel in valve["tunnels"]:
      graph[valve["name"]].append(tunnel)
  return graph


def find_path(graph, start, goal):
  explored = []
  queue = [[start]]
  if start == goal:
    return 0

  while queue:
    path = queue.pop(0)
    node = path[-1]

    if node not in explored:
      neighbors = graph[node]
      for neighbor in neighbors:
        new_path = list(path)
        new_path.append(neighbor)
        queue.append(new_path)

        if neighbor == goal:
          return len(new_path) - 1
      explored.append(node)
  return 'There is no path to connect these points'

def create_distances(nodes, graph):
  distances = {}
  for start in nodes:
    distances[start] = {}
    for goal in nodes:
      if start != goal and goal != "AA":
        distances[start][goal] = find_path(graph, start, goal) + 1  
  return distances

def create_valve_dictionary(valves):
  valve_dict = {}
  for valve in valves:
    valve_dict[valve["name"]] = valve
  return valve_dict

def find_optimal_pressure(node, tunnels, time_remaining, valves, distances):
  remaining_tunnels = tunnels.copy()
  remaining_tunnels.remove(node)
  if len(remaining_tunnels) == 0:
    return 0
  max_pressure = 0
  for tunnel in remaining_tunnels:
    if distances[node][tunnel] <= time_remaining:
      node_pressure = (time_remaining - distances[node][tunnel]) * valves[tunnel]["value"]
      all_pressure = node_pressure + find_optimal_pressure(tunnel, remaining_tunnels, time_remaining - distances[node][tunnel], valves, distances)
      if all_pressure > max_pressure:
        max_pressure = all_pressure
  return max_pressure



valves = parse_input(data)
nodes = find_important_nodes(valves)
graph = create_graph(valves)
distances = create_distances(nodes, graph)
print(find_optimal_pressure('AA', nodes, time_limit, create_valve_dictionary(valves), distances))

1737


In [85]:
import os
from collections import defaultdict

cwd = os.getcwd()
file = cwd + '/inputs/input16.txt'

data = open(file).read().splitlines()
time_limit = 26


def parse_input(data):
  valves = []
  for line in data:
    valve = { "name": line.split()[1]}
    valve["value"] = int(line.split(';')[0].split('=')[1])
    if line.find('valves') == -1:
      valve["tunnels"] = [line.split('valve')[1].strip()]
    else:
      tunnels = line.split('valves')[1].split(',')
      valve["tunnels"] = []
      for tunnel in tunnels:
        valve["tunnels"].append(tunnel.strip())
        
    valves.append(valve)
  return valves

def find_important_nodes(valves):
  important_nodes = ['AA']
  for valve in valves:
    if valve["value"] != 0:
      important_nodes.append(valve["name"])
  return important_nodes

def create_graph(valves):
  graph = {}
  for valve in valves:
    graph[valve["name"]] = []
    for tunnel in valve["tunnels"]:
      graph[valve["name"]].append(tunnel)
  return graph


def find_path(graph, start, goal):
  explored = []
  queue = [[start]]
  if start == goal:
    return 0

  while queue:
    path = queue.pop(0)
    node = path[-1]

    if node not in explored:
      neighbors = graph[node]
      for neighbor in neighbors:
        new_path = list(path)
        new_path.append(neighbor)
        queue.append(new_path)

        if neighbor == goal:
          return len(new_path) - 1
      explored.append(node)
  return 'There is no path to connect these points'

def create_distances(nodes, graph):
  distances = {}
  for start in nodes:
    distances[start] = {}
    for goal in nodes:
      if start != goal and goal != "AA":
        distances[start][goal] = find_path(graph, start, goal) + 1  
  return distances

def create_valve_dictionary(valves):
  valve_dict = {}
  for valve in valves:
    valve_dict[valve["name"]] = valve
  return valve_dict

def find_optimal_pressure(node, tunnels, time_remaining, valves, distances):
  remaining_tunnels = tunnels.copy()
  remaining_tunnels.remove(node)
  if len(remaining_tunnels) == 0:
    return 0
  max_pressure = 0
  for tunnel in remaining_tunnels:
    if distances[node][tunnel] <= time_remaining:
      node_pressure = (time_remaining - distances[node][tunnel]) * valves[tunnel]["value"]
      all_pressure = node_pressure + find_optimal_pressure(tunnel, remaining_tunnels, time_remaining - distances[node][tunnel], valves, distances)
      if all_pressure > max_pressure:
        max_pressure = all_pressure
  return max_pressure

def find_optimal_pressure_duo(node, tunnels, time_remaining, valves, distances, time_limit):
  testing_node = ''
  remaining_tunnels = tunnels.copy()
  remaining_tunnels.remove(node)
  if len(remaining_tunnels) == 0:
    return find_optimal_pressure('AA', remaining_tunnels + ['AA'], time_limit, valves, distances)
  max_pressure = 0
  for tunnel in remaining_tunnels:
    if distances[node][tunnel] <= time_remaining:
      node_pressure = (time_remaining - distances[node][tunnel]) * valves[tunnel]["value"]
      all_pressure = node_pressure + find_optimal_pressure_duo(tunnel, remaining_tunnels, time_remaining - distances[node][tunnel], valves, distances, time_limit)
      elephant_tunnels = remaining_tunnels.copy()
      elephant_tunnels.append('AA')
      elephant_tunnels.remove(tunnel)
      elephant_pressure = node_pressure + find_optimal_pressure('AA', elephant_tunnels, time_limit, valves, distances)
      compare_pressure = 0
      if all_pressure > elephant_pressure:
        compare_pressure = all_pressure
      else:
        compare_pressure = elephant_pressure
      if compare_pressure > max_pressure:
        max_pressure = compare_pressure
        testing_node = tunnel
  return max_pressure


valves = parse_input(data)
nodes = find_important_nodes(valves)
graph = create_graph(valves)
distances = create_distances(nodes, graph)
print(find_optimal_pressure_duo('AA', nodes, time_limit, create_valve_dictionary(valves), distances, time_limit))

1707
